<a href="https://colab.research.google.com/github/wenjunc2/PAHO-Drug-Data-Cleaning-Summer-2022/blob/main/3_0_Data_Mapping_for_Chile_1_(Write_drug_name).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Set up indexes for clean catagol drug names

In [ ]:
# read into clean catagol name
import pandas as pd
CleanDataBase = pd.read_excel('CleanDataBase.xlsx')
CatalogName = CleanDataBase['Catalog Name']
CatalogName = list(set(CatalogName))
CatalogName = [x for x in CatalogName if str(x) != 'nan']

CatalogNameCopy = CatalogName.copy()
# uppercase everything
CatalogNameU = []
for i in CatalogNameCopy:
  CatalogNameU.append(i.upper())

In [ ]:
CatalogNameU[:5]

['EMTRICITABINE 200 MG + TENOFOVIR 25 MG, TABLET, 30 TABLETS BOTTLE',
 'CYCLOPHOSPHAMIDE 1000 MG, POWDER FOR INJECTION, TBD',
 'FLUCONAZOLE 200 MG, TBD, TBD',
 'ACICLOVIR 250 MG, POWDER FOR INJECTION, VIAL, 5X1',
 'TACROLIMUS 5 MG, CAPSULE, 100 CAPSULES BOTTLE']

In [ ]:
# split the CatalogNameU
#from nltk.tokenize import WordPunctTokenizer  # this function split the string by space
#tokenizer = WordPunctTokenizer()

CatalogNameUS = []
for item in CatalogNameU:
  temp = item.split(",")
  CatalogNameUS.append(temp)

In [ ]:
CatalogNameUS[:5] # after split by ","

[['EMTRICITABINE 200 MG + TENOFOVIR 25 MG', ' TABLET', ' 30 TABLETS BOTTLE'],
 ['CYCLOPHOSPHAMIDE 1000 MG', ' POWDER FOR INJECTION', ' TBD'],
 ['FLUCONAZOLE 200 MG', ' TBD', ' TBD'],
 ['ACICLOVIR 250 MG', ' POWDER FOR INJECTION', ' VIAL', ' 5X1'],
 ['TACROLIMUS 5 MG', ' CAPSULE', ' 100 CAPSULES BOTTLE']]

Clean Catalog Drug Name - presentation form

In [ ]:
# summary the list of presentation form
PresentationForm = []

for item in CatalogNameUS:
  if len(item) >= 2:
    PresentationForm.append(item[1])

In [ ]:
PresentationForm = list(set(PresentationForm))

len(PresentationForm)

29

In [ ]:
PresentationForm[:5]# the presentation form

[' INHALER',
 ' SUSPENSION FOR INTRATRACHEAL INSTILLATION',
 ' TABLET (DISPERSIBLE',
 ' TABLET',
 ' TABLET (COATED)']

In [ ]:
PresentationDiction = []

from nltk.tokenize import WordPunctTokenizer  # this function split the string by space
tokenizer = WordPunctTokenizer()

for item in PresentationForm:
  tokenlist = tokenizer.tokenize(item)
  PresentationDiction.append(tokenlist)

In [ ]:
PresentationDiction2 = []

for item in PresentationDiction:
  for subitem in item:
    if len(subitem) > 3:
      PresentationDiction2.append(subitem)

In [ ]:
PresentationDiction2 = list(set(PresentationDiction2))

Clean Catalog Drug Name - main medicine : for get index

In [ ]:
# grab the main medicine 
MainMedicine = []

for item in CatalogNameUS:
  MainMedicine.append(item[0])

In [ ]:
print(len(MainMedicine), MainMedicine[:5])

462 ['EMTRICITABINE 200 MG + TENOFOVIR 25 MG', 'CYCLOPHOSPHAMIDE 1000 MG', 'FLUCONAZOLE 200 MG', 'ACICLOVIR 250 MG', 'TACROLIMUS 5 MG']


In [ ]:
# split MainMedicine to get only the medicne name, and use the medicine name as the index
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

MainMedicineSpl = []
for item in MainMedicine:
  tokenlist = tokenizer.tokenize(item)
  MainMedicineSpl.append(tokenlist)

In [ ]:
MainMedicineSpl[:5]

[['EMTRICITABINE', '200', 'MG', '+', 'TENOFOVIR', '25', 'MG'],
 ['CYCLOPHOSPHAMIDE', '1000', 'MG'],
 ['FLUCONAZOLE', '200', 'MG'],
 ['ACICLOVIR', '250', 'MG'],
 ['TACROLIMUS', '5', 'MG']]

In [ ]:
MainMedicineIndex = []
for item in MainMedicineSpl:
  templist = []
  for subitem in item:
    if len(subitem) > 4 and subitem.isdigit() is False:  ## ignoring these elements such as "MG", "+", ... 
      templist.append(subitem)
  MainMedicineIndex.append(templist)


In [ ]:
MainMedicineIndex[:5]

[['EMTRICITABINE', 'TENOFOVIR'],
 ['CYCLOPHOSPHAMIDE'],
 ['FLUCONAZOLE'],
 ['ACICLOVIR'],
 ['TACROLIMUS']]

In [ ]:
CatalogIndexData = {"Catalog Name": CatalogNameU, "Catalog Index": MainMedicineIndex}

In [ ]:
CatalogIndexData_df = pd.DataFrame(CatalogIndexData)

In [ ]:
len(CatalogIndexData_df)

462

In [ ]:
CatalogIndexData_df[:5]

,Catalog Name,Catalog Index
0,"EMTRICITABINE 200 MG + TENOFOVIR 25 MG, TABLET...","[EMTRICITABINE, TENOFOVIR]"
1,"CYCLOPHOSPHAMIDE 1000 MG, POWDER FOR INJECTION...",[CYCLOPHOSPHAMIDE]
2,"FLUCONAZOLE 200 MG, TBD, TBD",[FLUCONAZOLE]
3,"ACICLOVIR 250 MG, POWDER FOR INJECTION, VIAL, 5X1",[ACICLOVIR]
4,"TACROLIMUS 5 MG, CAPSULE, 100 CAPSULES BOTTLE",[TACROLIMUS]


List of main medicine which is used to compare and pair with Chile 1

In [ ]:
# create a list for main medicine from catalog name data set
CatalogMedName = []
for item in MainMedicineIndex:
  for subitem in item:
    CatalogMedName.append(subitem)

In [ ]:
CatalogMedName = list(set(CatalogMedName))
len(CatalogMedName)

206

In [ ]:
CatalogMedName[:3]

['SODIUM', 'NEVIRAPINE', 'MISOPROSTOL']

## Part 2: get the chile 1 data set drug names list

In [ ]:
import chardet
with open('CHLDiab1.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'confidence': 0.73, 'encoding': 'Windows-1252', 'language': ''}

In [ ]:
import pandas as pd
CHLDiab1 = pd.read_csv('CHLDiab1.csv',encoding='Windows-1252')
#CHLDiab1.head(1)

In [ ]:
CHLDiab1DirtyName = CHLDiab1['Descripcion linea Adquisicion']

In [ ]:
len(CHLDiab1DirtyName) # length of drug names

20606

In [ ]:
CHLDiab1DirtyName = list(set(CHLDiab1DirtyName))
len(CHLDiab1DirtyName) # length of drug names after remove duplicates

3663

In [ ]:
# uppercase everything
CHLDiab1DirtyNameU = []
for i in CHLDiab1DirtyName:
  CHLDiab1DirtyNameU.append(i.upper())

In [ ]:
CHLDiab1DirtyNameU[:5]

['INSULINA GLULISINA FA 100 UI/ML ULTRA RAPIDA APLICADOR INYECTABLE. DEBE INCLUIR AGUJAS PARA LÁPIZ',
 'SITAGLIPTINA + METFORMINA 50/850 MG COMP.',
 'INSULINA PARA COBAS E 411',
 'DAPAGLIFLOZINA 10 MG + METFORMINA 1000 MG. COMPRIMIDOS RECUBIERTOS DE LIBERACIÓN PROLONGADA, (CAJA DE 28 COMPRIMIDOS)',
 '2628739 INSULINA ASPARTICA ULTRARAPIDA 100 U/ML 3ML LAPIZ SOL INY']

In [ ]:
!pip install googletrans==3.1.0a0
from googletrans import Translator
translator = Translator(service_urls=['translate.googleapis.com'])
example = CHLDiab1DirtyNameU[0]
translation = translator.translate(example, dest='en')
print(translation.text)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.1 MB/s 
     |████████████████████████████████| 1.4 MB 8.9 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 65 kB 3.8 MB/s 
     |████████████████████████████████| 53 kB 2.3 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=01840923cca42e325ba2de557d5b5bf50d55977255a658f0ea402764bf26566c
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans
INSULIN GLUISINE FA 100 IU/ML ULTRA FAST INJECTABLE APPLICATOR. MUST INCLUDE PENCIL NEEDLES


In [ ]:
CHLDiab1DirtyNameU[0]

'INSULINA GLULISINA FA 100 UI/ML ULTRA RAPIDA APLICADOR INYECTABLE. DEBE INCLUIR AGUJAS PARA LÁPIZ'

In [ ]:
# get the orignila index about medicine name for each drug name 

from nltk.tokenize import WordPunctTokenizer  # this function split the string by space
tokenizer = WordPunctTokenizer()

CHLDiab1DirtyNameUSpl = []
for item in CHLDiab1DirtyNameU:
  tokenlist = tokenizer.tokenize(item)
  CHLDiab1DirtyNameUSpl.append(tokenlist)

In [ ]:
CHLDiab1DirtyNameUSpl[:1]

[['INSULINA',
  'GLULISINA',
  'FA',
  '100',
  'UI',
  '/',
  'ML',
  'ULTRA',
  'RAPIDA',
  'APLICADOR',
  'INYECTABLE',
  '.',
  'DEBE',
  'INCLUIR',
  'AGUJAS',
  'PARA',
  'LÁPIZ']]

In [ ]:
CHLDiab1DirtyNameIndex = []
for item in CHLDiab1DirtyNameUSpl:
  templist = []
  for subitem in item:
    if len(subitem) > 4 and subitem.isdigit() is False:  ## ignoring these elements such as "MG", "+", ... 
      templist.append(subitem)
  CHLDiab1DirtyNameIndex.append(templist)

In [ ]:
CHLDiab1DirtyNameIndex[:1]

[['INSULINA',
  'GLULISINA',
  'ULTRA',
  'RAPIDA',
  'APLICADOR',
  'INYECTABLE',
  'INCLUIR',
  'AGUJAS',
  'LÁPIZ']]

In [ ]:
# translate these names (indexes) to English
CHLDiab1DirtyNameIndexEng = []

for item in CHLDiab1DirtyNameIndex:
  temp_sublist = []
  for subitem in item:
    temp = translator.translate(subitem, dest='en')
    temp_sublist.append(temp.text)
  CHLDiab1DirtyNameIndexEng.append(temp_sublist)

# this block runs 21 mins

In [ ]:
# delete the information in each piece of drug name after SIMILAR
def DelSIMILAR(subitem):
  finalindex = len(subitem)
  for subindex in range(0, len(subitem)):
    if subitem[subindex] == 'SIMILAR':
      finalindex = subindex
  return subitem[:finalindex]

In [ ]:
# delete the information in each piece of drug name after EQUIVALENT
def DelEQUIVALENT(subitem):
  finalindex = len(subitem)
  for subindex in range(0, len(subitem)):
    if subitem[subindex] == 'EQUIVALENT':
      finalindex = subindex
  return subitem[:finalindex]

In [ ]:
CHLDiab1DirtyNameIndexEngDelSIMILAR = []
for item in CHLDiab1DirtyNameIndexEng:
  add = DelSIMILAR(item)
  add = DelEQUIVALENT(add)
  CHLDiab1DirtyNameIndexEngDelSIMILAR.append(add)

In [ ]:
# for the vocabulaies which are included in the medicine index get from clean data set, keep them in the 
# Chile 1 data index
def KeepMetAnd(subitem):
  addlist = []
  for element in subitem:
    if element in CatalogMedName:
      addlist.append(element)
  return addlist

In [ ]:
CHIfinIndex = []
for sublist in CHLDiab1DirtyNameIndexEngDelSIMILAR:
  add = KeepMetAnd(sublist)
  CHIfinIndex.append(add)

In [ ]:
# count the number of Chile index sub list which is null []
count = 0
for i in CHIfinIndex:
  if not i:
    count += 1

In [ ]:
count

447

In [ ]:
nullindexlist = []
for i in range(0, len(CHIfinIndex)):
  if not CHIfinIndex[i]:
    nullindexlist.append(i)

In [ ]:
!pip install -U spacy
!pip install scispacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.3 MB 3.8 MB/s 
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.0
    Uninstalling spacy-3.4.0:
      Successfully uninstalled spacy-3.4.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 1.6 MB/s 
     |████████████████████████████████| 6.0 MB 6.2 MB/s 
     |████████████████████████████████| 71 kB 9.2 MB/s 
     |████████████████████████████████| 13.5 MB 26.3 MB/s 
     |████████████████████████████████| 188 kB 75.5 MB/s 
     |████████████████████████████████| 10.1 MB 26.3 MB/s 
     |████████████████████████████████| 660 kB 4.5 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1

In [ ]:
## Install scispaCy models
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_sm-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_craft_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_jnlpba_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bionlp13cg_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_lg-0.4.0.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.6 MB 1.3 MB/s 
     |████████████████████████████████| 5.8 MB 3.7 MB/s 
  Created wheel for en-core-sci-sm: filename=en_core_sci_sm-0.4.0-py3-none-any.whl size=15660350 sha256=3a822088905798c722814558c25e9ed3749212591e1bb732c287fee407f01769
  Stored in directory: /root/.cache/pip/wheels/31/4b/3a/03f2732eb69789a7df3bf29c1bc010f660ada5f3c295d4073c
Successfully built en-core-sci-sm
  Attempting uninstall: typer
    Found existing installation: typer 0.4.2
    Uninstalling typer-0.4.2:
      Successfully uninstalled typer-0.4.2
  Attempting uninstall: spacy
    Found existing installation: spacy 3.2.4
    Uninstalling spacy-3.2.4:
      Successfully uninstalled spacy-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scisp

In [ ]:
import scispacy
import spacy

#Core models
import en_core_sci_sm
import en_core_sci_lg

#NER specific models
import en_ner_craft_md
import en_ner_bc5cdr_md
import en_ner_jnlpba_md
import en_ner_bionlp13cg_md

#Tools for extracting & displaying data
from spacy import displacy
import pandas as pd

In [ ]:
nullindexlist[:10]

[5, 10, 12, 15, 18, 19, 28, 29, 48, 53]

In [ ]:
CHLDiab1DirtyNameIndexEngDelSIMILAR[11]

['REQUIRES', 'ACQUIRE', 'METFORMIN', 'HYDROCHLORIDE', 'RELEASE', 'EXTENDED']

In [ ]:
#Load specific model and pass text through
nlp = en_core_sci_lg.load()

In [ ]:
#Load specific model and pass text through
#nlp = en_core_sci_lg.load()
#doc = nlp(CHLDiab1DirtyNameIndexEngDelSIMILAR3)

#Display resulting entity extraction
#displacy_image = displacy.render(doc, jupyter=True,style='ent')

an example for spacy

In [ ]:
CHLDiab1DirtyNameIndexEngDelSIMILAR[11]

['REQUIRES', 'ACQUIRE', 'METFORMIN', 'HYDROCHLORIDE', 'RELEASE', 'EXTENDED']

In [ ]:
CHLDiab1DirtyNameIndexEngDelSIMILAR11 = str(' '.join(CHLDiab1DirtyNameIndexEngDelSIMILAR[11]))

In [ ]:
doc = nlp(CHLDiab1DirtyNameIndexEngDelSIMILAR11)

#Display resulting entity extraction
displacy_image = displacy.render(doc, jupyter=True,style='ent')

In [ ]:
doc.text

'REQUIRES ACQUIRE METFORMIN HYDROCHLORIDE RELEASE EXTENDED'

for these drug names in Chile 1, which are not paired with clean index, use spacy nlp to choose the appropriate name

In [ ]:
Chile1drugOutDataName = []
for nullindex in nullindexlist:
  tempChiledrugname = CHLDiab1DirtyNameIndexEngDelSIMILAR[nullindex]
  tempChiledrugname_str = str(' '.join(tempChiledrugname))
  doc = nlp(tempChiledrugname_str)
  temptext = doc.text
  Chile1drugOutDataName.append(temptext)

In [ ]:
Chile1drugOutDataName[:10]

['SYRINGE island DELIVERY IMMEDIATE SKILLS',
 'GLIFORTEX',
 'DIAMICRON',
 'JANUMET 1000MG',
 'GLIFORTEX TABLETS',
 'JANUMET',
 'GLICLAZIDA TABLETS RELEASE EXTENDED CONTAINER CONTAINING TABLETS',
 'SAXAGLYPTINA TABLETS COATED PRESENTATION SALE PUBLIC CONVENIENT SINCE POINT VISTA PRECIO ANY LESS MONTHS EFFECTIVENESS MANUFACTURED CHILE CERTIFICATE COMPLIANCE GOOD PRACTICES MANUFACTURING ISSUED',
 'HIPOGLUCIN 850MG',
 'APIDRA SOLOSTAR']

there are still 12 drug names without index

In [ ]:
Chile1drugOutDataNameNullNumber = 0

for i in Chile1drugOutDataName:
  if not i:
    Chile1drugOutDataNameNullNumber += 1

In [ ]:
Chile1drugOutDataNameNullNumber

12

In [ ]:
Chile1drugOutDataNameNullIndex = []

for i in range(0, len(Chile1drugOutDataName)):
  if not Chile1drugOutDataName[i]:
    Chile1drugOutDataNameNullIndex.append(i)


In [ ]:
Chile1drugOutDataNameNullIndex

[20, 35, 55, 106, 113, 163, 343, 346, 369, 374, 395, 413]

In [ ]:
# for these null element in CHIfnIndex list, use Chile1drugOutDataName to complete them
j = 0
for i in nullindexlist:
  CHIfinIndex[i] = Chile1drugOutDataName[j]
  j += 1


In [ ]:
len(nullindexlist)

447

In [ ]:
for i in range(0, len(CHIfinIndex)):
  if not CHIfinIndex[i]:
    print(i)

131
229
385
775
812
1226
2747
2785
2989
3048
3274
3421


In [ ]:
for index in range(len(CHIfinIndex)):
  if not CHIfinIndex[index]:
    CHIfinIndex[index] = CHLDiab1DirtyNameIndexEngDelSIMILAR[index]

In [ ]:
nullindex = []
for index in range(len(CHIfinIndex)):
  if not CHIfinIndex[index]:
    nullindex.append(index)

In [ ]:
nullindex

[131, 229, 385, 775, 812, 1226, 2747, 2785, 2989, 3048, 3274, 3421]

check for the chile 1 drug index to get the presentation and weight

In [ ]:
ChilePreList = []

for item in CHLDiab1DirtyNameIndexEngDelSIMILAR:
  addlist = []
  for subitem in item:
    if subitem in PresentationDiction2:
      addlist.append(subitem)
  ChilePreList.append(list(set(addlist)))

In [ ]:
WeightList = []

for item in CHLDiab1DirtyNameUSpl:
  addlist = []
  for subitem in range(0, len(item)):
    if item[subitem].isdigit():
      if subitem + 1 < len(item):
        if item[subitem + 1].lower() in ['mg', 'ml']:
          addlist.append(item[subitem])
          addlist.append(item[subitem + 1])
  WeightList.append(list(set(addlist)))

In [ ]:
print(len(CHIfinIndex), len(ChilePreList), len(WeightList))

3663 3663 3663


In [ ]:
addsub

['INSULIN', []]

In [ ]:
# for catalog name index dataframe, transform list to string
str_CHIfinIndex = []
for item in CHIfinIndex:
  if not isinstance(item, str):
    addsub = []
    for subitem in item:
      if subitem:
        addsub.append(subitem)
        newitem = ' '.join(addsub)
  else:
    newitem = item

  str_CHIfinIndex.append(newitem)

In [ ]:
str_CHIfinIndex[:4]

['INSULIN', 'SITAGLIPTIN METFORMIN', 'INSULIN', 'METFORMIN']

In [ ]:
str_ChilePreList = []

for item in ChilePreList:
  newitem = ' '.join(item)
  str_ChilePreList.append(newitem)

In [ ]:
str_WeightList = []

for item in WeightList:
  newitem = ' '.join(item)
  str_WeightList.append(newitem)

In [ ]:
Chile1DrugDataWrite = {"Final Drug Index": str_CHIfinIndex, "Presentation Form": str_ChilePreList, "Weight": str_WeightList}

In [ ]:
Chile1DrugDataWrite_df = pd.DataFrame(Chile1DrugDataWrite)

In [ ]:
Chile1DrugDataWrite_df

,Final Drug Index,Presentation Form,Weight
0,INSULIN,INJECTABLE,
1,SITAGLIPTIN METFORMIN,,850 MG
2,INSULIN,,
3,METFORMIN,COATED RELEASE,10 1000 MG
4,INSULIN,,
...,...,...,...
3658,INSULIN RECOMBINANT,,
3659,METFORMIN,RELEASE,500 MG
3660,METFORMIN,,850 MG
3661,EMPAGLIFLOZIN METFORMIN,,850 MG


In [ ]:
Chile1DrugDataWrite_df.to_excel("Chile1DrugDataWrite.xlsx", sheet_name = 'Chile1DrugDataWrite')  